In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

# build our own SparkSession
spark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1682798761495_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
bucket_name = "pmbdgrupo15" #adjust to your case
prefix_aws = "s3a://"+bucket_name+"/dataset/" #choose the folder where your dataset lives
prefix_out_aws = "s3a://"+bucket_name+"/dataout/" #choose a created folder where you want to store data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load data

## Uber data

In [ ]:
uber_full_data = spark.read.parquet(prefix_aws + "parquets")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
uber_full_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [ ]:
uber_full_data.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [ ]:
uber_full_data = uber_full_data.withColumn('datetime', F.to_date('request_datetime'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Weather data

In [ ]:
weather = spark.read.csv(prefix_aws + 'nyc 2021-01-01 to 2021-12-31.csv', inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
weather.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- resolvedAddress: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- feelslike: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- precip: double (nullable = true)
 |-- precipprob: integer (nullable = true)
 |-- preciptype: string (nullable = true)
 |-- snow: double (nullable = true)
 |-- snowdepth: double (nullable = true)
 |-- windgust: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- winddir: double (nullable = true)
 |-- sealevelpressure: double (nullable = true)
 |-- cloudcover: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- uvindex: integer (nullable = true)
 |-- severerisk: string (nullable = true)

In [ ]:
weather.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+--------------------+----------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-------+----------+
|name|address|     resolvedAddress|  datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|uvindex|severerisk|
+----+-------+--------------------+----------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+-------+----------+
| nyc|    nyc|New York, NY, Uni...|2021-01-01| 2.5|     -0.2|-3.0|    67.8| 15.33|       100|      rain| 0.0|      0.0|    null|     15.5|   69.8|          1028.9|      50.6|      14.0|      3|      null|
| nyc|    nyc|New York, NY, Uni...|2021-01-02| 5.8|      3.6| 1.2|    74.0|  2.38|       100|      rain| 1.9|      0.6|    54.6|     25.5|  246.9|          1012.4|      63.9|      

In [ ]:
weather_count = weather.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# print('\nNulls in dataframe:')
# cols_to_forget = [c for c in weather.columns if isinstance(weather.schema[c].dataType, TimestampType)]
# weather_cols_interest = [x for x in weather.columns if x not in cols_to_forget]
# for cl in weather_cols_interest:
#     k = weather.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
#     if k > 0:
#         print(f'Column {cl} with {k} nulls or NaN, out of {weather_count} records ({k*100/weather_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# print('\nNulls in dataframe:')
# cols_to_check = [c for c in weather.columns if isinstance(weather.schema[c].dataType, TimestampType)]

# for cl in cols_to_check:
#     k = weather.select(cl).filter(F.col(cl).isNull()).count()
#     if k > 0:
#         print(f'Column {cl} with {k} nulls or NaN, out of {weather_count} records ({k*100/weather_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_weather = weather.select('datetime', 'temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow', 'snowdepth', 'windspeed', 'cloudcover', 'visibility', 'uvindex', F.to_date('datetime').alias('day'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_weather = new_weather.withColumn('quarter', F.date_format('datetime', 'Q'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_weather = new_weather.drop('datetime')
new_weather = new_weather.select('day', 'quarter', 'temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow', 'snowdepth', 'windspeed', 'cloudcover', 'visibility', 'uvindex')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_weather = new_weather.withColumnRenamed('day', 'datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
new_weather.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+----+---------+----+--------+------+----------+----+---------+---------+----------+----------+-------+
|  datetime|quarter|temp|feelslike| dew|humidity|precip|precipprob|snow|snowdepth|windspeed|cloudcover|visibility|uvindex|
+----------+-------+----+---------+----+--------+------+----------+----+---------+---------+----------+----------+-------+
|2021-01-01|      1| 2.5|     -0.2|-3.0|    67.8| 15.33|       100| 0.0|      0.0|     15.5|      50.6|      14.0|      3|
|2021-01-02|      1| 5.8|      3.6| 1.2|    74.0|  2.38|       100| 1.9|      0.6|     25.5|      63.9|      12.2|      5|
|2021-01-03|      1| 2.5|     -1.6|-0.5|    80.7|  5.09|       100| 1.2|      1.7|     24.1|      81.5|      13.2|      1|
|2021-01-04|      1| 3.6|      1.1|-0.2|    76.6|  0.84|       100| 0.5|      0.7|     17.3|      89.3|      15.6|      4|
|2021-01-05|      1| 3.8|      1.3|-1.5|    68.7|   0.0|         0| 0.0|      0.1|     15.2|      98.8|      16.0|      2|
+----------+----

## Taxi zones

In [ ]:
taxi_zones = spark.read.csv(prefix_aws + 'taxi_zone_lookup.csv', inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
taxi_zones.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

In [ ]:
taxi_zones.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [ ]:
taxi_zones.groupBy('service_zone').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|service_zone|count|
+------------+-----+
|         N/A|    2|
|   Boro Zone|  205|
| Yellow Zone|   55|
|         EWR|    1|
|    Airports|    2|
+------------+-----+

In [ ]:
taxi_zones = taxi_zones.drop('service_zone')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Join

In [ ]:
join_df = uber_full_data.join(new_weather, 'datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
join_df = join_df.join(taxi_zones, join_df['PULocationID'] == taxi_zones['LocationID'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
join_df = join_df.drop('LocationID')
join_df = join_df.withColumnRenamed('Borough', 'PUBorough')
join_df = join_df.withColumnRenamed('Zone', 'PUZone')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
join_df = join_df.join(taxi_zones, join_df['DOLocationID'] == taxi_zones['LocationID'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
join_df = join_df.drop('LocationID')
join_df = join_df.withColumnRenamed('Borough', 'DOBorough')
join_df = join_df.withColumnRenamed('Zone', 'DOZone')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
join_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+-------+----+---------+----+--------+------+----------+----+---------+---------+----------+----------+-------+---------+--------------------+---------+--------------------+
|  datetime|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|quarter|temp|feelslike| dew|humidity|precip|precipprob|snow|snowdepth|w

# Pré-processamento

In [ ]:
pre_process_df = join_df.alias('pre_process_df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pre_process_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- datetime: date (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true

In [ ]:
pre_process_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+-------+----+---------+----+--------+------+----------+----+---------+---------+----------+----------+-------+---------+--------------------+---------+--------------------+
|  datetime|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|quarter|temp|feelslike| dew|humidity|precip|precipprob|snow|snowdepth|w

In [ ]:
pre_process_df_count = pre_process_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print('\nNulls in dataframe:')
cols_to_forget = ["datetime", "request_datetime", "pickup_datetime", "dropoff_datetime", "on_scene_datetime"]
df_cols_interest = [x for x in pre_process_df.columns if x not in cols_to_forget]
for cl in df_cols_interest:
    k = pre_process_df.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
       print(f'Column {cl} with {k} nulls or NaN, out of {pre_process_df_count} records ({k*100/pre_process_df_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in dataframe:
Column originating_base_num with 48429333 nulls or NaN, out of 174596164 records (27.74%)
Column airport_fee with 36495849 nulls or NaN, out of 174596164 records (20.90%)

In [ ]:
print('\nNulls in dataframe:')
cols_to_check = ["datetime", "request_datetime", "pickup_datetime", "dropoff_datetime", "on_scene_datetime"]

for cl in cols_to_check:
    k = pre_process_df.select(cl).filter(F.col(cl).isNull()).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {pre_process_df_count} records ({k*100/pre_process_df_count:.2f}%)')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Nulls in dataframe:
Column on_scene_datetime with 48408476 nulls or NaN, out of 174596164 records (27.73%)

In [ ]:
# Substituir valores para o nome da empresa
pre_process_df = pre_process_df.withColumn('Hvfhs_license_num', 
                   F.when(pre_process_df['Hvfhs_license_num'] == 'HV0003', 'Uber')
                   .when(pre_process_df['Hvfhs_license_num'] == 'HV0004', 'Via')
                   .when(pre_process_df['Hvfhs_license_num'] == 'HV0005', 'Lyft')
                   .when(pre_process_df['Hvfhs_license_num'] == 'HV0002', 'Juno')
                   .otherwise(pre_process_df['Hvfhs_license_num'])
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Existem 5 registos a null ou Na, que iremos descartar
pre_process_df = pre_process_df.na.drop(subset=['request_datetime']) 

# Criação da variável hour
pre_process_df = pre_process_df.withColumn('hour', F.hour(F.col('request_datetime')))

# Dividir a coluna request_datetime em data e hora
pre_process_df = pre_process_df.withColumn('request_date', F.split(pre_process_df['request_datetime'], ' ')[0])
pre_process_df = pre_process_df.withColumn('request_time', F.split(pre_process_df['request_datetime'], ' ')[1])

# Converter as colunas em tipos de dados de data e hora
pre_process_df = pre_process_df.withColumn('request_date', F.to_date(pre_process_df['request_date']))
pre_process_df = pre_process_df.withColumn('request_time', F.date_format('request_time', 'HH:mm:ss'))

pre_process_df = pre_process_df.drop('request_datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Dividir a coluna request_datetime em data e hora
pre_process_df = pre_process_df.withColumn('pickup_date', F.split(pre_process_df['pickup_datetime'], ' ')[0])
pre_process_df = pre_process_df.withColumn('pickup_time', F.split(pre_process_df['pickup_datetime'], ' ')[1])

# Converter as colunas em tipos de dados de data e hora
pre_process_df = pre_process_df.withColumn('pickup_date', F.to_date(pre_process_df['pickup_date']))
pre_process_df = pre_process_df.withColumn('pickup_time', F.date_format('pickup_time', 'HH:mm:ss'))

# Descartar a coluna pickup_datatime
pre_process_df = pre_process_df.drop('pickup_datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Criar a coluna waiting_time
pre_process_df = pre_process_df.withColumn('request_timestamp', F.to_timestamp(F.col('request_time'), 'HH:mm:ss'))
pre_process_df = pre_process_df.withColumn('pickup_timestamp', F.to_timestamp(F.col('pickup_time'), 'HH:mm:ss'))
pre_process_df = pre_process_df.withColumn('waiting_time_seconds', F.unix_timestamp(F.col('pickup_timestamp')) - F.unix_timestamp(F.col('request_timestamp')))
pre_process_df = pre_process_df.withColumn('waiting_time', F.from_unixtime(F.col('waiting_time_seconds'), 'HH:mm:ss'))

pre_process_df = pre_process_df.drop('request_timestamp')
pre_process_df = pre_process_df.drop('pickup_timestamp')
pre_process_df = pre_process_df.drop('waiting_time_seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Eliminar a coluna on_scene_datetime
pre_process_df = pre_process_df.drop('on_scene_datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Dividir a coluna dropoff_datetime em data e hora
pre_process_df = pre_process_df.withColumn('dropoff_date', F.split(pre_process_df['dropoff_datetime'], ' ')[0])
pre_process_df = pre_process_df.withColumn('dropoff_time', F.split(pre_process_df['dropoff_datetime'], ' ')[1])

# Converter as colunas em tipos de dados de data e hora
pre_process_df = pre_process_df.withColumn('dropoff_date', F.to_date(pre_process_df['dropoff_date']))
pre_process_df = pre_process_df.withColumn('dropoff_time', F.date_format('dropoff_time', 'HH:mm:ss'))

pre_process_df = pre_process_df.drop('dropoff_datetime')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Criar a nova coluna "trip_km" e descartar linhas com trip_miles nulo
pre_process_df = pre_process_df.na.drop(subset=['trip_miles'])  # Descartar linhas com trip_miles nulo
pre_process_df = pre_process_df.withColumn('trip_km', F.round(F.col('trip_miles') * 1.60934, 2))  # Converter milhas para km e arredondar para 2 casas decimais

pre_process_df = pre_process_df.drop('trip_miles')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Substitui todos os valores null por 0
pre_process_df = pre_process_df.fillna({'airport_fee': 0}, subset=['airport_fee'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Crie a coluna "client_total_paid_without_tips" somando os valores das outras colunas
pre_process_df = pre_process_df.withColumn(
    "client_total_paid_without_tips",
    F.round(
        F.coalesce(F.col("base_passenger_fare"), F.lit(0))
        + F.coalesce(F.col("tolls"), F.lit(0))
        + F.coalesce(F.col("bcf"), F.lit(0))
        + F.coalesce(F.col("sales_tax"), F.lit(0))
        + F.coalesce(F.col("congestion_surcharge"), F.lit(0))
        + F.coalesce(F.col("airport_fee"), F.lit(0)),
        2  # número de casas decimais
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Crie a coluna "client_total_paid" que será a variável target
pre_process_df = pre_process_df.withColumn(
    "client_total_paid",
    F.round(
        F.coalesce(F.col("base_passenger_fare"), F.lit(0))
        + F.coalesce(F.col("tolls"), F.lit(0))
        + F.coalesce(F.col("bcf"), F.lit(0))
        + F.coalesce(F.col("sales_tax"), F.lit(0))
        + F.coalesce(F.col("congestion_surcharge"), F.lit(0))
        + F.coalesce(F.col("airport_fee"), F.lit(0))
        + F.coalesce(F.col("tips"), F.lit(0)),
        2  # número de casas decimais
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# substituir os N e espaços vazios por 0 e 1 na tabela "access_a_ride_flag"
pre_process_df = pre_process_df.withColumn('access_a_ride_flag_binary', F.when(F.col('access_a_ride_flag') == 'N', 0).otherwise(1))

pre_process_df = pre_process_df.drop('access_a_ride_flag')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# cria a coluna chosen_first
pre_process_df = pre_process_df.withColumn('chosen_first', 
                   F.when(F.col('originating_base_num').isNull(), 2)  # Se originating_base_num for null, atribuir 2
                   .when(F.col('originating_base_num') == F.col('dispatching_base_num'), 1)  # Se as colunas forem iguais, atribuir 1
                   .otherwise(0)  # Caso contrário, atribuir 0
                  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Criar a nova coluna "shared_request_flag_binary"
pre_process_df = pre_process_df.withColumn('shared_request_flag_binary', F.when(F.col('shared_request_flag') == 'N', 0).otherwise(1))
pre_process_df = pre_process_df.drop('shared_request_flag')

# Criar a nova coluna "shared_match_flag_binary"
pre_process_df = pre_process_df.withColumn('shared_match_flag_binary', F.when(F.col('shared_match_flag') == 'N', 0).otherwise(1))
pre_process_df = pre_process_df.drop('shared_match_flag')

# Criar a nova coluna "wav_request_flag_binary"
pre_process_df = pre_process_df.withColumn('wav_request_flag_binary', F.when(F.col('wav_request_flag') == 'N', 0).otherwise(1))
pre_process_df = pre_process_df.drop('wav_request_flag')

# Criar a nova coluna "wav_match_flag_binary"
pre_process_df = pre_process_df.withColumn('wav_match_flag_binary', F.when(F.col('wav_match_flag') == 'N', 0).otherwise(1))
pre_process_df = pre_process_df.drop('wav_match_flag')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pre_process_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+--------------------+--------------------+------------+------------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------+----+---------+----+--------+------+----------+----+---------+---------+----------+----------+-------+---------+--------------------+---------+--------------------+----+------------+------------+-----------+-----------+------------+------------+------------+-------+------------------------------+-----------------+-------------------------+------------+--------------------------+------------------------+-----------------------+---------------------+
|  datetime|Hvfhs_license_num|dispatching_base_num|originating_base_num|PULocationID|DOLocationID|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|quarter|temp|feelslike| dew|humidity|precip|precipprob|snow|snowdepth|windspeed|cloudcover|visibility|uvindex|PUBorough|              PUZone|DO

In [ ]:
# Crie a coluna "client_tips_percentage" indica o valor que o condutor recebe sem incluir as gorjetas
pre_process_df = pre_process_df.withColumn("client_tips_percentage", F.round((F.col("tips") * 100)/F.col("client_total_paid_without_tips"), 2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Criação da coluna apenas com o mês
pre_process_df = pre_process_df.withColumn('month_col', F.month(pre_process_df['datetime']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pre_process_df = pre_process_df.drop("originating_base_num")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pre_process_df = pre_process_df.fillna({'client_tips_percentage': 0}, subset=['client_tips_percentage'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# pre_process_df = pre_process_df.withColumn("datetime", F.col("datetime").cast("integer"))
# pre_process_df = pre_process_df.withColumn("Hvfhs_license_num", F.col("Hvfhs_license_num").cast("integer"))
# pre_process_df = pre_process_df.withColumn("dispatching_base_num", F.col("dispatching_base_num").cast("integer"))
# pre_process_df = pre_process_df.withColumn("quarter", F.col("quarter").cast("integer"))
# pre_process_df = pre_process_df.withColumn("PUBorough", F.col("PUBorough").cast("integer"))
# pre_process_df = pre_process_df.withColumn("PUZone", F.col("PUZone").cast("integer"))
# pre_process_df = pre_process_df.withColumn("DOBorough", F.col("DOBorough").cast("integer"))
# pre_process_df = pre_process_df.withColumn("DOZone", F.col("DOZone").cast("integer"))
# pre_process_df = pre_process_df.withColumn("request_date", F.col("request_date").cast("integer"))
# pre_process_df = pre_process_df.withColumn("request_time", F.col("request_time").cast("integer"))
# pre_process_df = pre_process_df.withColumn("pickup_date", F.col("pickup_date").cast("integer"))
# pre_process_df = pre_process_df.withColumn("pickup_time", F.col("pickup_time").cast("integer"))
# pre_process_df = pre_process_df.withColumn("waiting_time", F.col("waiting_time").cast("integer"))
# pre_process_df = pre_process_df.withColumn("dropoff_date", F.col("dropoff_date").cast("integer"))
# pre_process_df = pre_process_df.withColumn("dropoff_time", F.col("dropoff_time").cast("integer"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# pre_process_df.write.parquet(prefix_out_aws + "clean_data.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# EDA - Exploratory Data Analysis

In [ ]:
df_license = pre_process_df.groupBy('hvfhs_license_num').count()
df_license.write.parquet(prefix_out_aws + "df_license.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_dispatching = pre_process_df.groupby("dispatching_base_num").count().orderBy('count', ascending=False)
df_dispatching.write.parquet(prefix_out_aws + 'dispatching.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_PUBorough = pre_process_df.groupby("PUBorough").count()
df_PUBorough.write.parquet(prefix_out_aws + "PUBorough.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_DOBorough = pre_process_df.groupby("DOBorough").count()
df_DOBorough.write.parquet(prefix_out_aws + 'DOBorough.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_shared_request = pre_process_df.groupby("shared_request_flag_binary").count().orderBy('count', ascending=False)
df_shared_request.write.parquet(prefix_out_aws + 'shared_request.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_shared_match = pre_process_df.groupby("shared_match_flag_binary").count().orderBy('count', ascending=False)
df_shared_match.write.parquet(prefix_out_aws + 'shared_match.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_request_flag = pre_process_df.groupby("wav_request_flag_binary").count().orderBy('count', ascending=False)
df_request_flag.write.parquet(prefix_out_aws + 'request_flag.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_wav_match = pre_process_df.groupby("wav_match_flag_binary").count().orderBy('count', ascending=False)
df_wav_match.write.parquet(prefix_out_aws + "wav_match.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_trip_km = pre_process_df.select("trip_km")
df_trip_km.write.parquet(prefix_out_aws + "trip_km.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_client_total_paid = pre_process_df.select("client_total_paid")
df_client_total_paid.write.parquet(prefix_out_aws + "total_paid.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_trips_month = pre_process_df.groupby("month_col").count()
df_trips_month.write.parquet(prefix_out_aws + "monthly_trips.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_daily_trips = pre_process_df.groupby("datetime").count()
df_daily_trips.write.parquet(prefix_out_aws + "daily_trips.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_hourly_trips = pre_process_df.groupby("hour").count()
df_hourly_trips.write.parquet(prefix_out_aws + "hourly_trips.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_origem_destino = pre_process_df.select("PUBorough", "DOBorough")
df_origem_destino.write.parquet(prefix_out_aws + "origem_destino.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.stat import Correlation
# input_cols = pre_process_df.columns
# assembler = VectorAssembler(inputCols=input_cols, outputCol='features') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# vec_df = assembler.transform(pre_process_df).select('features')
# corr_matrix = Correlation.corr(vec_df, input_cols).collect()[0][0].toArray().tolist()
# corr_matrix.write.parquet(prefix_out_aws + "corr_matrix.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Modeling

https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

## Imports e divisão dados

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
input_cols = [
 'PULocationID',
 'DOLocationID',
 'trip_time',
 'base_passenger_fare',
 'tolls',
 'bcf',
 'sales_tax',
 'congestion_surcharge',
 'airport_fee',
 'tips',
 'driver_pay',
 'temp',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'snow',
 'snowdepth',
 'windspeed',
 'cloudcover',
 'visibility',
 'uvindex',
 'trip_km',
 'client_total_paid_without_tips',
 'access_a_ride_flag_binary',
 'chosen_first',
 'shared_request_flag_binary',
 'shared_match_flag_binary',
 'wav_request_flag_binary',
 'wav_match_flag_binary',
 'month_col']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
train_data, test_data = pre_process_df.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
vec_train_df = assembler.transform(train_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
vec_test_df = assembler.transform(test_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Regressão linear

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='client_total_paid')
lr_model = lr.fit(vec_train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions = lr_model.transform(vec_test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='client_total_paid')

lr_r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
lr_rmse = evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'})
lr_mae = evaluator.evaluate(predictions, {evaluator.metricName: 'mae'})
lr_mse = evaluator.evaluate(predictions, {evaluator.metricName: 'mse'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print('R-squared (R2):', lr_r2)
print('Root Mean Squared Error (RMSE):', lr_rmse)
print('Mean Absolute Error (MAE):', lr_mae)
print('Mean Squared Error (MSE):', lr_mse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R-squared (R2): 1.0
Root Mean Squared Error (RMSE): 1.307232599367845e-09
Mean Absolute Error (MAE): 6.597651943457023e-10
Mean Squared Error (MSE): 1.708857068850013e-18

In [ ]:
print(lr_model.summary)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Árvores de decisão

In [ ]:
dt = DecisionTreeRegressor(featuresCol='features', labelCol='client_total_paid')
dt_model = dt.fit(vec_train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions = dt_model.transform(vec_test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='client_total_paid')

dt_r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
dt_rmse = evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'})
dt_mae = evaluator.evaluate(predictions, {evaluator.metricName: 'mae'})
dt_mse = evaluator.evaluate(predictions, {evaluator.metricName: 'mse'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print('R-squared (R2):', dt_r2)
print('Root Mean Squared Error (RMSE):', dt_rmse)
print('Mean Absolute Error (MAE):', dt_mae)
print('Mean Squared Error (MSE):', dt_mse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R-squared (R2): 0.9073704163800763
Root Mean Squared Error (RMSE): 6.820621925117062
Mean Absolute Error (MAE): 1.936737729118179
Mean Squared Error (MSE): 46.52088344538758

In [ ]:
print(dt_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_f3d1f0ddddd1, depth=5, numNodes=63, numFeatures=32

## Random forest

In [ ]:
rf = RandomForestRegressor(featuresCol='features', labelCol='client_total_paid')
rf_model = rf.fit(vec_train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions = rf_model.transform(vec_test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='client_total_paid')

rf_r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
rf_rmse = evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'})
rf_mae = evaluator.evaluate(predictions, {evaluator.metricName: 'mae'})
rf_mse = evaluator.evaluate(predictions, {evaluator.metricName: 'mse'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print('R-squared (R2):', rf_r2)
print('Root Mean Squared Error (RMSE):', rf_rmse)
print('Mean Absolute Error (MAE):', rf_mae)
print('Mean Squared Error (MSE):', rf_mse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R-squared (R2): 0.90737394573139
Root Mean Squared Error (RMSE): 6.820491984987165
Mean Absolute Error (MAE): 1.8104961080079292
Mean Squared Error (MSE): 46.51911091727416

In [ ]:
print(rf_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForestRegressionModel: uid=RandomForestRegressor_46d665ef3998, numTrees=20, numFeatures=32